In [45]:
import json
import os
import numpy
import pandas
import pyarrow
import sys

from datetime import date
from os import path
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

In [2]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
import create_queries
%load_ext autoreload
%autoreload 2

In [8]:
COMPANY_NAME = 'HPCC'
COMPANY_IDENTIFIER = 'HPCC'
TRANSFER_PACKAGES_START_DATE = '2019-01-01'
SALES_TRANSACTIONS_START_DATE = '2019-01-01'
INVENTORY_DATES = [
    '09/30/2020',
    '10/31/2020',
    '11/30/2020',
    '12/31/2020',
    '01/31/2021',
    '02/28/2021',
    '03/31/2021',
    '04/30/2021',
    '05/31/2021',
    '06/30/2021',
    '07/31/2021',
    '08/31/2021',
    '09/30/2021',
]
ANALYSIS_PARAMS = {
    'sold_threshold': 1.0
}
TODAY_DATE = date.today().strftime('%m/%d/%Y')
INVENTORY_DATES.append(TODAY_DATE)
print('Today is {}'.format(TODAY_DATE))

Today is 10/28/2021


In [46]:
# Download packages, sales transactions, incoming / outgoing tranfers

company_licenses_query = create_queries.create_company_licenses_query(COMPANY_IDENTIFIER)
company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_sales_transactions_query = create_queries.create_company_sales_transactions_query(COMPANY_IDENTIFIER, SALES_TRANSACTIONS_START_DATE)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(
    COMPANY_IDENTIFIER, include_quantity_zero=True)

company_licenses_dataframe = pandas.read_sql_query(company_licenses_query, engine)
company_incoming_transfer_packages_dataframe = pandas.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pandas.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_sales_transactions_dataframe = pandas.read_sql_query(company_sales_transactions_query, engine)
company_inventory_packages_dataframe = pandas.read_sql_query(company_inventory_packages_query, engine)

In [47]:
company_licenses_dataframe

,us_state,license_number,license_category,legal_name,is_current,license_status,rollup_id,license_description
0,CA,C10-0000005-LIC,Retailer,"DPC SF, LLC",True,Active,CA-01004,Retailer
1,CA,C10-0000064-LIC,Retailer,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",True,Active,CA-01805,Retailer


In [23]:
missing_received_datetime_incoming_transfer_packages_dataframe = company_incoming_transfer_packages_dataframe[company_incoming_transfer_packages_dataframe.received_datetime.isnull()]
missing_received_datetime_incoming_transfer_packages_dataframe[[
    'delivery_type',
    'manifest_number',
    'created_date',
    'received_datetime',
    'shipment_type_name',
    'shipment_transaction_type',
    'shipment_package_state',
    'product_category_name',
    'product_name',
    'shipper_wholesale_price',
    'shipped_quantity',
    'shipped_unit_of_measure',
    'received_quantity',
    'received_unit_of_measure',
]]

,delivery_type,manifest_number,created_date,received_datetime,shipment_type_name,shipment_transaction_type,shipment_package_state,product_category_name,product_name,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure
0,INCOMING_FROM_VENDOR,0002635732,2021-10-28,NaT,Wholesale Manifest,Wholesale,Shipped,Shake (Packaged Eighth - each),Strong Ag Coastal Sun Chem Kardashian 1/8 Display,0.01,1.0,Each,NaN,None
1,INCOMING_FROM_VENDOR,0002637701,2021-10-28,NaT,Wholesale Manifest,Wholesale,Shipped,Edible (weight - each),14288 Kikoko Mints 40-pack Sleep Little Helper...,162.00,12.0,Each,NaN,None
2,INCOMING_FROM_VENDOR,0002637701,2021-10-28,NaT,Wholesale Manifest,Wholesale,Shipped,Edible (weight - each),15076 Kikoko Mints 20-pack Boost Little Helper...,180.00,12.0,Each,NaN,None
3,INCOMING_FROM_VENDOR,0002633609,2021-10-28,NaT,Wholesale Manifest,Wholesale,Shipped,Vape Cartridge (weight - each),Honeyleaf - Original Glue - 0.5 - 0.5g,192.00,32.0,Each,NaN,None
4,INCOMING_FROM_VENDOR,0002633609,2021-10-28,NaT,Wholesale Manifest,Wholesale,Shipped,Vape Cartridge (weight - each),STIIIZY - Strawnana Pod - 0.5g,200.00,20.0,Each,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5298,INCOMING_FROM_VENDOR,0002265932,2021-08-02,NaT,Wholesale Manifest,Wholesale,Shipped,Edible (weight - each),Froot Sour Gummy Watermelon 100mg,350.00,100.0,None,NaN,None
5299,INCOMING_FROM_VENDOR,0002265932,2021-08-02,NaT,Wholesale Manifest,Wholesale,Shipped,Edible (weight - each),Froot Gummies Orange 100mg,400.00,100.0,None,NaN,None
5525,INCOMING_FROM_VENDOR,0002247265,2021-07-28,NaT,Wholesale Manifest,Wholesale,Shipped,Flower (packaged - each),ENDOS Flower Formula 1 -3.5g,0.01,1.0,None,NaN,None
5537,INCOMING_FROM_VENDOR,0002247265,2021-07-28,NaT,Wholesale Manifest,Wholesale,Shipped,Pre-Roll Infused,ENDOS Infused Pre-Roll 1g Watermelon Kush,0.01,1.0,None,NaN,None


In [26]:
print(f'Filtering out incoming transfer packages with "Shipped" shipment package state...')
print(f'Before filter # of incoming transfer packages: {len(company_incoming_transfer_packages_dataframe.index)}')
company_incoming_transfer_packages_dataframe = company_incoming_transfer_packages_dataframe[
    company_incoming_transfer_packages_dataframe['shipment_package_state'] != 'Shipped'
]
print(f'After filter # of incoming transfer packages: {len(company_incoming_transfer_packages_dataframe.index)}')

Filtering out incoming transfer packages with "Shipped" shipment package state...
Before filter # of incoming transfer packages: 47871
After filter # of incoming transfer packages: 47871


In [27]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts/analysis")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from util import active_inventory_util as util

In [28]:
d = util.Download()
d.download_dataframes(
    incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
    outgoing_transfer_packages_dataframe=company_outgoing_transfer_packages_dataframe,
    sales_transactions_dataframe=company_sales_transactions_dataframe,
)

In [29]:
import importlib
importlib.reload(util)

# TODO(dlluncor): Just for debugging to make this faster
INVENTORY_DATES = [TODAY_DATE]

date_to_inventory_packages_dataframe = {}
id_to_history = util.get_histories(d)

for inventory_date in INVENTORY_DATES:
    computed_inventory_package_records = util.create_inventory_dataframe_by_date(
        id_to_history,
        inventory_date,
        params=ANALYSIS_PARAMS,
    )
    computed_inventory_packages_dataframe = pandas.DataFrame(
        computed_inventory_package_records,
        columns=[
            'package_id',
            'license_number',
            'arrived_date',
            'product_category_name',
            'product_name',
            'quantity',
            'sold_date',
            'is_in_inventory'
        ]
    )
    date_to_inventory_packages_dataframe[inventory_date] = computed_inventory_packages_dataframe

WARN: package #5172609 does not have a shipped quantity
WARN: package #848108 does not have a shipped quantity


In [31]:
from_packages_inventory_dataframe = company_inventory_packages_dataframe[[
    'package_id',
    'packaged_date',
    'product_category_name',
    'product_name',
    'quantity',
]].sort_values('package_id')

package_id_to_actual_row = {}
for index, row in from_packages_inventory_dataframe.iterrows():
    package_id_to_actual_row[str(row['package_id'])] = row

In [60]:
import importlib
importlib.reload(util)

res = util.compare_inventory_dataframes(
    computed=date_to_inventory_packages_dataframe[TODAY_DATE],
    actual=from_packages_inventory_dataframe,
    options={
        'num_errors_to_show': 20,
        'accept_computed_when_sold_out': True
    }
)

# Two categories for: Num actual packages not computed: 229
# 1. Actual has a package that we've never seen
# 2. Actual has a package that we've sold out of, and that's why it's in the inventory

# Two categories for: Num computed packages not in actual: 237
# 1. We never saw the package in the actual inventory
# 2. Actual inventory is sold out, but we think it's not sold out in computed

Pct of # inventory matching: 89.83% (2041 / 2272)
Accuracy of quantities: 92.12%
Pct of # inventory packages over-estimated: 586.84%
Pct of # quantity over-estimated: 5.65%
Avg quantity delta: 0.80
Avg quantity: 10.16

Num matching packages: 2041
Num actual packages not computed: 231
  but computed at some point: 0, e.g., 0.00% of non-computed packages
  avg quantity from actual packages 0.00
Num computed packages not in actual: 13333
  but in actual inventory at some point: 19

Computed has these extra package IDs; first 20
766095; computed quantity 7500
766099; computed quantity 1830
1725817; computed quantity 899
1725818; computed quantity 899
367796; computed quantity 896
369009; computed quantity 892
367795; computed quantity 672
3753529; computed quantity 550
4001831; computed quantity 469
366378; computed quantity 457
607314; computed quantity 454
369011; computed quantity 448
1416017; computed quantity 420
107389; computed quantity 350
366379; computed quantity 344
606997; comp

In [61]:
# For debugging individual package histories
# You have to run the above block to reload the package_id_to_history array

import importlib
importlib.reload(util)

# Missing transactions?
# 5189649

# Using grams or missing transactions?
#WARN: seeing an incoming package for #19083030 with no received_datetime
#WARN: seeing an incoming package for #19087962 with no received_datetime
#WARN: seeing an incoming package for #19083120 with no received_datetime
#WARN: seeing an incoming package for #19083028 with no received_datetime
#Exception: FATAL error, could not find a transfer to insert a tx with date
#2021-10-22 11:58:45+00:00 into for package 18218608

PACKAGE_IDS = [
    '107500'
]

util.analyze_specific_package_histories(
    d, package_id_to_actual_row, PACKAGE_IDS, params=ANALYSIS_PARAMS)

! Missing in metrc_packages
{'package_row_id': 'd51e91b0-02a9-4661-876a-5b1ed0b71940', 'delivery_type': 'INCOMING_FROM_VENDOR', 'license_number': 'C10-0000005-LIC', 'manifest_number': '0000028123', 'created_date': datetime.date(2019, 7, 5), 'received_datetime': Timestamp('2019-07-05 21:24:26+0000', tz='UTC'), 'shipper_facility_license_number': 'C11-0000039-LIC', 'shipper_facility_name': 'Humnoldt Farms-- CCAP #062 (Horizon LGG LLC)', 'recipient_facility_license_number': 'C10-0000005-LIC', 'recipient_facility_name': 'DPC SF, LLC', 'shipment_type_name': 'Transfer', 'shipment_transaction_type': 'Standard', 'package_id': '107500', 'package_label': '1A40603000001F5000001207', 'type': 'transfer', 'shipment_package_state': 'Accepted', 'is_testing_sample': False, 'is_trade_sample': False, 'product_category_name': 'Flower', 'product_name': 'Classic Death Star Cherry Pie Eighth- Humboldt Farms', 'package_lab_results_status': 'failed', 'shipper_wholesale_price': nan, 'shipped_quantity': 336.0, 's

In [62]:
inactive_packages_df = pandas.read_sql_query(
    util.are_packages_inactive_query(res['computed_extra_package_ids']),
    engine
)

In [63]:
inactive_packages_df[inactive_packages_df['package_id'] == '107500']

,identifier,license_number,type,package_id,package_label,product_category_name,product_name,archiveddate,finisheddate,quantity
6311,HPCC,C10-0000005-LIC,inactive,107500,1A40603000001F5000001207,Flower,Classic Death Star Cherry Pie Eighth- Humboldt...,None,2019-11-27,0.0
